# Программирование для всех<br>(основы работы с Python)

*Тамбовцева А.А.*

## Практикум 6.1. Введение в парсинг HTML

### Постановка задачи

Есть реально существующая [страница](https://elementy.ru/problems/3181/Sekrety_tablitsy_Pifagora) со статьей по математике для школьников.

Исходный код данной страницы достаточно громоздкий из-за отсылок к стилевым файлам, фрагментам кода JavaScript для реализации разных процессов и дополнительной информации, которую в режиме пользователя мы не видим. Поэтому была написана упрощенная [версия](https://html-preview.github.io/?url=https://github.com/allatambov/PyAll25/blob/main/page0601.html) этой страницы. В этой версии убраны элементы, не относящиеся непосредственно к статье, плюс, без специальных настроек формулы не отображаются красиво. Это нормально, так должно и быть.

Чтобы увидеть исходный код упрощенной версии страницы для изучения и отправки запроса, используйте следующую ссылку: https://raw.githubusercontent.com/allatambov/PyAll25/refs/heads/main/page0601.html.

Для выполнения заданий необходимо ознакомиться с полным исходным кодом страницы, инструменты разработчика здесь не сработают, поскольку упрощенная версия страницы лежит на Github как обычный текст. Другими словами, нужно просто посмотреть, что в каких тэгах находится, сопоставляя [это](https://html-preview.github.io/?url=https://github.com/allatambov/PyAll25/blob/main/page0601.html) и [это](https://raw.githubusercontent.com/allatambov/PyAll25/refs/heads/main/page0601.html).

### Подготовка к работе

Импортируем необходимые библиотеки. Нам понадобится модуль `requests` для отправки запросов, для «подключения» к странице и получения её содержимого в виде строки, и функция `BeautifulSoup` из библиотеки `bs4` для удобного поиска по полученной строке:

In [ ]:
import requests
from bs4 import BeautifulSoup

Подключаемся к странице сайта, ссылка на которую сохранена в переменной `link`:

In [ ]:
link = "https://raw.githubusercontent.com/allatambov/PyAll25/refs/heads/main/page0601.html"
page = requests.get(link)
print(page)

Объект, который мы сохранили в `page`, имеет особый тип `requests.models.Response`, он же ответ на запрос:

In [ ]:
print(type(page))

Сам по себе объект полностью мы увидеть не можем, но из него можно извлечь разные атрибуты – характеристики объекта. Например, статус ответа, который мы и так увидели выше:

In [ ]:
print(page.status_code)

Код 200 означает, что запрос получен, обработан, и ответ на него получен. Другими словами, у нас нет проблем с доступом к странице (потенциальная проблема со стороны сервера, например, ведущиеся технические работы или отказ в правах доступа) и нет проблем с самим запросом (потенциальная проблема со стороны пользователя, например, «битая» ссылка). Про коды статусов можно почитать в [статье](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).

Аналогичным образом можем узнать кодировку страницы:

In [ ]:
print(page.encoding)

И, самое главное, весь исходный код HTML в виде большой строки с текстом:

In [ ]:
print(type(page.text))
print(page.text)

Выполнять поиск по такой строке с тэгами не очень удобно (даже если вы знакомы с регулярными выражениями), поэтому преобразуем строку в объект типа `BeautifulSoup`. Такой объект внешне несильно отличается от обычной строки, однако внутри по структуре похож на словарь, и это сходство значительно упрощает поиск по тэгам и атрибутам.

In [ ]:
soup = BeautifulSoup(page.text)
print(type(soup))
print(soup)

### Примеры поиска по тэгам

На объектах `BeautifulSoup` определены методы `.find()` и `.find_all()`. Оба метода возвращают фрагменты кода HTML, которые соответствуют критериям поиска, различие заключается в том, что метод `.find()` предназначен для поиска одного совпадения (если критериям поиска соответствует несколько элементов на странице, то берётся только первый), а метод `.find_all()` – для поиска всех совпадений. В первом случае результат возвращается в виде одного элемента типа `BeautifulSoup`, а во втором – в виде списка таких элементов. Давайте попробуем что-то поискать!

Найдем заголовок статьи. Если внимательно посмотреть на код страницы, заметим, что он находится внутри тэга `<h1>`. Так как заголовок у статьи обычно один, задействуем метод `.find()`, который возвращает один результат:

In [ ]:
soup.find("h1")

Результат – фрагмент кода HTML со всеми тэгами. Как получить «чистый» текст? Запросить текст с помощью атрибута `.text`:

In [ ]:
title = soup.find("h1").text
print(title)

Найдем теперь подзаголовки статьи – здесь это *Задача*, *Подсказка*, *Решение* и *Послесловие*. Если вновь внимательно посмотрим на исходный код страницы, заметим, что они находятся в тэгах `<h2>`. Так как их несколько, задействуем метод `.find_all()`, который возвращает все результаты поиска списком:

In [ ]:
soup.find_all("h2")

Каждый элемент списка – объект типа `BeautifulSoup` с тэгами, придется забрать атрибут `.text` из каждого элемента списка через списковое включение:

In [ ]:
raw_headings = soup.find_all("h2")
headings = [h.text for h in raw_headings]
print(headings)

### Задача 1

Выполнив поиск по объекту `soup`, найдите и сохраните в список **все абзацы** с текстом статьи (без заголовков, подзаголовков, дополнительных подписей). Элементы этого списка должны быть обычными строками с «чистым» текстом без тэгов.

Объедините элементы списка в одну строку с полным текстом статьи, сохраните его в переменную `text`.

In [ ]:
### YOUR CODE HERE ###

### Задача 2

Выполнив поиск по объекту `soup`, любым способом найдите и сохраните в переменную `author` **имя и фамилию автора** статьи (без лишних пробелов и символов в строке).

In [ ]:
### YOUR CODE HERE ###

### Задача 3

Выполнив поиск по объекту `soup`, любым способом найдите и сохраните в переменную `date` **дату** статьи.

In [ ]:
### YOUR CODE HERE ###

### Задача 4

Объедините информацию, полученную в предыдущих задачах, в словарь вида:

    {"title" : значение1,
     "heads" : значение2, 
     "text" : значение3, 
     "author" : значение4, 
     "date" : значение5},
     
где вместо `значение1`-`значение5` подставлены значения из переменных `title`, `headings` `text`, `author` и `date`, созданных ранее.

In [ ]:
### YOUR CODE HERE ###

### Задача 5

Найдите в исходном тексте все фрагменты с тэгом `<mark>`. Извлеките текст и составьте слово – сохраните его в переменную :)

In [ ]:
### YOUR CODE HERE ###